<a href="https://colab.research.google.com/github/raduga256/Text-Classification-with-NLP-Tf-Idf-vs-Word2Vec-vs-BERT/blob/main/news_category_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## New Category Text Classification with NLP: Tf-Idf vs Word2Vec vs BERT

There are different techniques to extract information from raw text data and use it to train a classification model. This tutorial compares the old school approach of Bag-of-Words (used with a simple machine learning algorithm), the popular Word Embedding model (used with a deep learning neural network), and the state of the art Language models (used with transfer learning from attention-based transformers) that have completely revolutionized the NLP landscape.

This article yourself first appearance paper of LIME in certain seminars "Why Should I Trust You?": Explaining the Predictions of Any Classifierare based on a slide that was announced.
https://irisu-inwl.github.io/reveal/lime

However, because this slide there is a feeling that was made for those who do not know the machine learning, it was summarized as anew article.

In [2]:
!pip install lime

     |████████████████████████████████| 276kB 20.9MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-cp37-none-any.whl size=283846 sha256=b9d5e91993612107b4ef8fd7a0549d8265318131da74508206eea14096f0fd52
  Stored in directory: /root/.cache/pip/wheels/4c/4f/a5/0bc765457bd41378bf3ce8d17d7495369d6e7ca3b712c60c89
Successfully built lime


In [4]:
 # Installing the Tranformers Library
 !pip -qq install transformers

     |████████████████████████████████| 2.0MB 17.6MB/s 
     |████████████████████████████████| 870kB 47.3MB/s 
     |████████████████████████████████| 3.2MB 51.8MB/s 


In [5]:
# First of all, I need to import the following libraries:

## for data
import json
import pandas as pd
import numpy as np

## for plotting
import matplotlib.pyplot
import seaborn as sns

## for processing
import re
import nltk

## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing

## for explainer
from lime import lime_text

## for word embedding
import gensim
import gensim.downloader as gensim_api

## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

## for bert language model
import transformers
